In [26]:
import os
import sys
import pandas as pd
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import XMLParser,ParseError
import numpy as np
import skimage.draw
import skimage.io


parser = ET.XMLParser(encoding="iso-8859-5") # Parser for XML
# Getting neccessary directories
dir_name= os.fspath('C:\\Users\\HP USER\\Documents\\Drone Datasets\\semantic_drone_dataset\\training_set')
image_dir=os.path.join(dir_name,'images')
annotations=os.path.join(dir_name,'gt\\semantic\\label_me_xml')
bounding_box=os.path.join(dir_name,'gt\bounding_box\label_me_xml')
mask_dir=os.path.join(dir_name,'gt\\semantic\\label_images')
class_rgb=os.path.join(dir_name,'gt\\semantic\\class_dict.csv')
tree= os.path.join(annotations,'004.xml')
#Reading the files
class_rgb = pd.read_csv(class_rgb,delimiter=',') #The class and RGB values for the masks in the final output

#The XML file handling
tree = ET.parse(tree)
root = tree.getroot()

child_tag=[]
for child in root:
    child_tag.append((child.tag, child.attrib))
elem_tag=[elem.tag for elem in root.iter()]
xml_string=ET.tostring(root, encoding='utf8').decode('utf8')
#print(xml_string)
#SI_attrib=[sourceImage.attrib for sourceImage in root.iter("sourceImage")]
#SI_text=[sourceImage.text for sourceImage in root.iter("sourceImage")]
#Fname_text=[filename.text for filename in root.iter("filename")]




        

In [20]:
# Tryig to create the imageids from the xml files, not adviced.
image_ids=[]
for item in ["%03d" % i for i in range(1,599)]: # Creating 3-digits figures to suit the xml file naming
    try:
        tree= os.path.join(annotations,str(item)+'.xml')
        tree = ET.parse(tree)
        root = tree.getroot()
        file_name_text=[filename.text for filename in root.iter("filename")]
        file_name_text=", ".join(map(str, file_name_text))  #Removing the bracket before appending to image_ids
        image_ids.append(file_name_text)
    except (FileNotFoundError,ParseError):
        pass
    
len(image_ids)


398

In [14]:

class droneDataset(utils.Dataset):
    def load_drone(self, dataset_dir, subset):
        class_name=class_rgb['name'].tolist()
        for i,classes in enumerate(class_name[1:23],1):
            self.add_class("semantic", i, "classes")
        
        #Which subset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        # Get image ids from directory names        
        image_ids = next(os.walk(dataset_dir))[2]
        if subset == "val":
            image_ids= list(set(image_ids[350:]))
        else:
            if subset == "train":
                image_ids = list(set(image_ids) - set(image_ids[350:]))

        # Add images
        for image_id in image_ids:
            self.add_image(
                "semantic",
                image_id=image_id,
                path=os.path.join(dataset_dir, image_id, "images/{}.jpg".format(image_id)))

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        # Get mask directory from image path
        mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "masks")
        # Read mask files from .png image
        mask = []
        for f in next(os.walk(mask_dir))[2]:
            if f.endswith(".png"):
                m = skimage.io.imread(os.path.join(mask_dir, f)).astype(np.bool)
                mask.append(m)
        mask = np.stack(mask, axis=-1)

In [19]:
for item in ["%03d" % i for i in range(1,599)]: # Creating 3-digits figures to suit the xml file naming
    key= "parser" + str(item)
    

In [27]:
mask = []
for f in next(os.walk(mask_dir))[2]:
    if f.endswith(".png"):
        m =skimage.io.imread(os.path.join(mask_dir, f)).astype(np.bool)
        mask.append(m)
#mask = np.stack(mask, axis=-1)
mask

MemoryError: 